<a href="https://colab.research.google.com/github/bhushanbrb/test/blob/master/CRF_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 3.8MB/s 


In [0]:
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [7]:
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


True

In [8]:
['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [0]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))


In [0]:
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))


In [27]:
print(train_sents[0])

type(train_sents)

type(train_sents[0][0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


tuple

In [38]:
tmp = train_sents[0]

print(tmp)
type(tmp)

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


list

In [56]:
tmp[0][1][:2]

'NP'

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag,
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
           # '-1:postag': postag1,
           # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
       # postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
          #  '+1:postag': postag1,
          #  '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [65]:
sent2features(train_sents[0])[0]


{'+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': '(',
 'BOS': True,
 'bias': 1.0,
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'melbourne',
 'word[-2:]': 'ne',
 'word[-3:]': 'rne'}

In [75]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 934 ms, sys: 216 ms, total: 1.15 s
Wall time: 1.15 s


In [149]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.30343585910230,
    c2=0.06146442424219609,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 34.4 s, sys: 28.4 ms, total: 34.4 s
Wall time: 34.5 s


In [0]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [148]:

str_updated = [('John', '', 'B-PER'), ('lives', '', 'O'), ('in', 'Fpt', 'O'), ('America', '', 'B-LOC')]

str_test = sent2features(str_updated)
str_label = sent2labels(str_updated)
print(str_test)
print(str_label)


[{'bias': 1.0, 'word.lower()': 'john', 'word[-3:]': 'ohn', 'word[-2:]': 'hn', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'BOS': True, '+1:word.lower()': 'lives', '+1:word.istitle()': False, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'lives', 'word[-3:]': 'ves', 'word[-2:]': 'es', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, '-1:word.lower()': 'john', '-1:word.istitle()': True, '-1:word.isupper()': False, '+1:word.lower()': 'in', '+1:word.istitle()': False, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'in', 'word[-3:]': 'in', 'word[-2:]': 'in', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, '-1:word.lower()': 'lives', '-1:word.istitle()': False, '-1:word.isupper()': False, '+1:word.lower()': 'america', '+1:word.istitle()': True, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'america', 'word[-3:]': 'ica', 'word[-2:]': 'ca', 'word.isupper()': False, 'word.ist

In [147]:
crf.predict([str_test])

[['O', 'O', 'O', 'B-LOC']]

In [117]:
len(X_test)

type(X_test)

print(X_test[0])

[{'bias': 1.0, 'word.lower()': 'la', 'word[-3:]': 'La', 'word[-2:]': 'La', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'BOS': True, '+1:word.lower()': 'coruña', '+1:word.istitle()': True, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': 'coruña', 'word[-3:]': 'uña', 'word[-2:]': 'ña', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, '-1:word.lower()': 'la', '-1:word.istitle()': True, '-1:word.isupper()': False, '+1:word.lower()': ',', '+1:word.istitle()': False, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': ',', 'word[-3:]': ',', 'word[-2:]': ',', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, '-1:word.lower()': 'coruña', '-1:word.istitle()': True, '-1:word.isupper()': False, '+1:word.lower()': '23', '+1:word.istitle()': False, '+1:word.isupper()': False}, {'bias': 1.0, 'word.lower()': '23', 'word[-3:]': '23', 'word[-2:]': '23', 'word.isupper()': False, 'word.istitle()': False, 'wo

In [120]:
y_pred = crf.predict(X_test)

print(y_pred)


[['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'], ['O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'], ['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O'], ['B-PER', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'], ['O'], ['O', '

In [0]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(sorted_labels)

In [130]:
sorted(labels, key= lambda x: x[2])

['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']

In [133]:


print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=2
))

              precision    recall  f1-score   support

       B-LOC       0.80      0.78      0.79      1084
       I-LOC       0.67      0.63      0.65       325
      B-MISC       0.72      0.53      0.61       339
      I-MISC       0.69      0.58      0.63       557
       B-ORG       0.80      0.82      0.81      1400
       I-ORG       0.85      0.78      0.81      1104
       B-PER       0.83      0.87      0.85       735
       I-PER       0.88      0.94      0.91       634

   micro avg       0.80      0.78      0.79      6178
   macro avg       0.78      0.74      0.76      6178
weighted avg       0.80      0.78      0.79      6178

